## Extending the POD-Galerkin RB method for approximate transfer function evaluation to the IRKA framework

The following matrix computations must be done to construct the projection matrices $V$ and $W$ for the Iterative rational Krylov algorithm (IRKA):
\begin{equation*}
    (-\mu_{i}I_{n} - A)^{-1}B\hat{b}_{i}, \quad \text{and} \quad (-\mu_{i}I_{n} - A)^{-*}C^{T}\hat{c}_{i} \quad \text{for } i = 1,\ldots,r,
\end{equation*}
where $-\mu_{i}, \hat{c}_{i}, \hat{b}_{i}$ are interpolation data and $0 < r \leq n$ is the desired order of approximating ROM. 

So, we have decided to solve two parametrized linear coercive models to construct projection matrices $V$ and $W$:
\begin{equation}
    a_{1}(v_{1}, w; s) = l_{1}(w) \quad \text{and} \quad a_{2}(v_{2}, w; s) = l_{2}(w),
\end{equation}
where $a_{1}(v_{1}, w; s) = w^{*}(sI_{n} - A)v_{1}$ and $l_{1}(w) = w^{*}B$, and $a_{2}(v_{2}, w; s) = w^{*}(sI_{n} - A)^{*}v_{2}$ and $l_{2}(w) = w^{*}C^{T}$, and solutions to these parametrized linear coercive models are
\begin{equation*}
    v_{1}(s) = (sI_{n} - A)^{-1}B \quad \text{and} \quad v_{2}(s) = (sI_{n} - A)^{-*}C^{T}.
\end{equation*}
Therefore, knowing $v_{1}(\mu_{i})$ and $v_{2}(\mu_{i})$ for $i = 1, \ldots, r$ will suffice for constructing the projection matrices $V$ and $W$. Also, note that these two FOMs are parameter-separable, i.e.,
\begin{equation*}
    a_{1}(v_{1},w;s) = w^{*}(sI_{n} - A)v_{1} = sw^{*}I_{n}v_{1} - w^{*}Av_{1}  \quad  a_{2}(v_{2},w;s) = w^{*}(sI_{n} - A)^{*}v_{2} = \overline{s}w^{*}I_{n}v_{2} - w^{*}A^{*}v_{2}.
\end{equation*}

In [11]:
import numpy as np
import matplotlib.pyplot as plt

from pymor.basic import *
from pymor.models.basic import StationaryModel
from pymor.algorithms.to_matrix import to_matrix
from pymor.vectorarrays.numpy import NumpyVectorSpace
from pymor.operators.numpy import NumpyMatrixOperator
from pymor.operators.constructions import LincombOperator
from pymor.parameters.functionals import ProjectionParameterFunctional, ConjugateParameterFunctional

In [12]:
def MatrixModel(A, B, C):
    
    '''
    Description
    -----------
    This function creates a StationaryModel for the following linear coercive models derived from the given three matrices A, B, and C:
    
        a_1(v, w; s) = w*(sI_n - A)v and l_1(w) = w*B
        a_2(v, w; s) = w*(sI_n - A)*v and l_2(w) = w*C^T.

    Parameters
    ----------
    A: NumpyMatrixOperator or numpy.ndarray
        A.shape = (n, n) or to_matrix(A).shape = (n, n)
    B: NumpyMatrixOperator or numpy.ndarray
        A.shape = (n, 1) or to_matrix(B).shape = (n, 1)
    C: NumpyMatrixOperator or numpy.ndarray
        A.shape = (1, n) or to_matrix(C).shape = (1, n)

    Return
    ------
    model_V: StationaryModel
    model_W: StationaryModel
    '''

    # Define operators (and also a dimension of a model)
    if isinstance(A, np.ndarray):
        dim = A.shape[0]
        A_op = NumpyMatrixOperator(A)
    else:
        dim = to_matrix(A).shape[0]
        A_op = A

    if isinstance(B, np.ndarray):
        B_op = NumpyMatrixOperator(B) 
    else:
        B_op = B  

    if isinstance(C, np.ndarray):
        C_op = NumpyMatrixOperator(C.T)  
    else:
        C_op = C.H  # C is real, so adjoint is transpose

    I_op = NumpyMatrixOperator(np.eye(dim))
    
    # Define parameter functional for 's'
    s_param = ProjectionParameterFunctional('s', 1)

    # Define bilinear form a(v, w; s) = w*(sI - A)v
    a_op_1 = LincombOperator([I_op, A_op], [s_param, -1])

    # Define bilinear form a(v, w; s) = w*(sI - A)*v -> Note: (sI - A)* = s*I - A*
    a_op_2 = LincombOperator([I_op, A_op.H], [ConjugateParameterFunctional(s_param), -1])
    
    # Define linear functional l(w) = w^*B
    l_op_1 = B_op

    # Define linear functional l(w) = w^*C^{T}
    l_op_2 = C_op

    # Define the StationaryModels
    model_V = StationaryModel(operator=a_op_1, rhs=l_op_1)
    model_W = StationaryModel(operator=a_op_2, rhs=l_op_2)

    return [model_V, model_W] 


In [13]:
def MatrixReductor(model_V, model_W, training_set, reduced_order_V: int, reduced_order_W: int):
    
    '''
    Inputs:
    ------------------------------------------------
    model_V - Stationary Model of linear coercive model w*(sI_n - A)v = w*B -> StationaryModel
    model_W - Stationary Model of linear coercive model w*(sI_n - A)*v -> StationaryModel
    training_set - an array containing parameters used to construct the snapshot matrix -> type(training_set[i]) = pymor.parameters.base.Mu (list of Mu objects)
    ------------------------------------------------
    Outputs:
    ------------------------------------------------
    pod_rom_V
    pod_rom_W
    '''

    # Compute FOM solutions for the parameters in the training set
    solution_snapshots_V = model_V.solution_space.empty()
    solution_snapshots_W = model_W.solution_space.empty()
    for s in training_set:
        solution_snapshots_V.append(model_V.solve(s))
        solution_snapshots_W.append(model_W.solve(s))
        
    # Snapshot matrices
    snapshot_matrix_V = solution_snapshots_V.to_numpy().T # Note: One may also use solution_snapshots_V.impl._array.T to get np.ndarray type needed for computation
    snapshot_matrix_W = solution_snapshots_W.to_numpy().T

    # Finding the Singular Value Decomposition (SVD) of snapshot matrices -> S = UΣV^T
    U_V, D_V, Vt_V = np.linalg.svd(snapshot_matrix_V, full_matrices = True)
    U_W, D_W, Vt_W = np.linalg.svd(snapshot_matrix_W, full_matrices = True)

    if reduced_order_V > min(snapshot_matrix_V.shape):
        raise ValueError("'reduced_order_V' cannot exceed the rank of the snapshot matrix.")
    if reduced_order_W > min(snapshot_matrix_W.shape):
        raise ValueError("'reduced_order_W' cannot exceed the rank of the snapshot matrix.")

    # The reduced bases (POD bases)
    pod_basis_numpy_V = U_V[:,:reduced_order_V]
    pod_basis_numpy_W = U_W[:,:reduced_order_W]

    # Convert NumPy array into VectorArray 
    space_V = NumpyVectorSpace(model_V.order) #number of columns = model_V.order
    space_W = NumpyVectorSpace(model_W.order)
    pod_basis_V = space_V.make_array(pod_basis_numpy_V.T) #This is actually transpose of POD-RB basis
    pod_basis_W = space_W.make_array(pod_basis_numpy_W.T) #This is actually transpose of POD-RB basis
    
    # POD-Galerkin RB method
    pod_reductor_V = StationaryRBReductor(model_V, RB = pod_basis_V) 
    pod_reductor_W = StationaryRBReductor(model_W, RB = pod_basis_W) 
    pod_rom_V = pod_reductor_V.reduce()
    pod_rom_W = pod_reductor_W.reduce()

    return [pod_rom_V, pod_reductor_V, pod_rom_W, pod_reductor_W]

Let $A\in\mathbb{R}^{n\times n}$ $B\in\mathbb{R}^{n\times 1}$ $C\in\mathbb{R}^{1\times n}$ be given. The `ProjectionMatrices` function defined below includes `R_V = R^{V}_N(s)` and `R_W = R^{W}_M(s)`, which are given as:
\begin{equation*}
R^{V}_N(s) = 
\begin{bmatrix}
\hat{v}_{N}(s_{1}) & \hat{v}_{N}(s_{2}) &  \cdots  &\hat{v}_{N}(s_{r})
\end{bmatrix}
\quad \text{and} \quad
R^{W}_M(s) = 
\begin{bmatrix}
\hat{w}_{M}(s_{1}) & \hat{w}_{M}(s_{2}) & \cdots & \hat{w}_{M}(s_{r})
\end{bmatrix},
\end{equation*}
where $\hat{v}_{N}(s_{i})\in\mathbb{C}^{n\times 1}$ and $\hat{w}_{M}(s_{i})\in\mathbb{C}^{n\times 1}$ are the reconstructed solutions of the reduced solutions $v_{N}(s_{i})\in\mathbb{C}^{N\times 1}$ and $w_{M}(s_{i})\in\mathbb{C}^{M\times 1}$ with reduction orders $N$ and $M$, respectively, for the following (parameter-separable) parameterized linear coercive models (FOM) evaluated at a given parameter $s_{i}$.
\begin{align*}
    a_{1}(v, u; s) = l_{1}(u) \quad v, u\in \mathbb{C}^{n\times 1}\\
    a_{2}(w, u; s) = l_{2}(u) \quad w, u\in \mathbb{C}^{n\times 1}
\end{align*}
where $a_{1}(v, u; s) = u^{*}(sI_{n} - A)v\in \mathbb{C}$ and $l_{1}(u) = u^{*}B$, and $a_{2}(w, u; s) = u^{*}(sI_{n} - A)^{*}w\in \mathbb{C}$ and $l_{2}(u) = u^{*}C^{T}$, and solutions to these parametrized linear coercive models are
\begin{equation*}
    v(s) = (sI_{n} - A)^{-1}B \quad \text{and} \quad w(s) = (sI_{n} - A)^{-*}C^{T}.
\end{equation*}
Consider some initial interpolation data $-\mu_{i}, \hat{c}_{i}, \hat{b}_{i}\in\mathbb{C}$ for $0 < r \leq n$. Therefore, the POD-projection matrices are given by
\begin{equation*}
    V_{POD}= 
    \begin{bmatrix}
    \hat{v}_{N}(-\mu_{1})\hat{b}_{1}&\hat{v}_{N}(-\mu_{2})\hat{b}_{2}&\cdots&\hat{v}_{N}(-\mu_{r})\hat{b}_{r}
    \end{bmatrix} = R^{V}_N(\mu)D_{\hat{b}} \quad \text{and} \quad
    W_{POD} = 
    \begin{bmatrix}
    \hat{w}_{N}(-\mu_{1})\hat{c}_{1}&\hat{w}_{N}(-\mu_{2})\hat{c}_{2}&\cdots&\hat{w}_{N}(-\mu_{r})\hat{c}_{r}
    \end{bmatrix} = R^{W}_M(\mu)D_{\hat{c}},
\end{equation*}
where $\mu = (-\mu_{1}, -\mu_{2},\ldots, -\mu_{r})$ and 
\begin{equation*}
    D_{\hat{b}} = diag(\hat{b}_{1}, \hat{b}_{2}, \ldots, \hat{b}_{r}) \quad \text{and} \quad D_{\hat{c}} = diag(\hat{c}_{1}, \hat{c}_{2}, \ldots, \hat{c}_{r}).
\end{equation*}   
This observation provides us with a fairly good error estimate for the projection matrices.
\begin{align*}
    \lVert V - V_{POD}\rVert &= \lVert R^{V}(\mu)D_{\hat{b}} - R^{V}_N(\mu)D_{\hat{b}}\rVert = \lVert (R^{V}(\mu) - R^{V}_N(\mu))D_{\hat{b}}\rVert \leq \lVert D_{\hat{b}}\rVert \cdot \lVert R^{V}(\mu) - R^{V}_N(\mu)\rVert,\\
    \lVert W - W_{POD}\rVert &= \lVert R^{W}(\mu)D_{\hat{c}} - R^{W}_M(\mu)D_{\hat{c}}\rVert = \lVert (R^{W}(\mu) - R^{W}_M(\mu))D_{\hat{c}}\rVert \leq \lVert D_{\hat{c}}\rVert \cdot \lVert R^{W}(\mu) - R^{W}_M(\mu)\rVert.
\end{align*}  
This demonstrates that if $\lVert R^{V}(\mu) - R^{V}_N(\mu) \rVert$ and $\lVert R^{W}(\mu) - R^{W}_M(\mu) \rVert \to 0$ as $N, M \to n$, then $\lVert V - V_{POD} \rVert$ and $\lVert W - W_{POD} \rVert\to 0$.

In [14]:
def ProjectionMatrices(pod_rom_V, pod_reductor_V, pod_rom_W, pod_reductor_W, mu, b, c, biorth = None, numpy = None):

    '''
    Inputs:
    ------------------------------------------------
    pod_rom_V
    pod_rom_W
    pod_reductor_V
    pod_reductor_W
    mu - list of -mu_i values -> type(mu[i]) = pymor.parameters.base.Mu (list of Mu objects)
    b - NumPy array -> b.shape = (r,) where r = len(mu)
    c - NumPy array -> c.shape = (r,) where r = len(mu)
    validation_set - an array containing parameters used to evaluate the reduced model after its construction -> type(validation_set[i]) = pymor.parameters.base.Mu (list of Mu objects)
    ------------------------------------------------
    Outputs: Biorthonormal pair of projection matrices V, W using biorthonormal Gram-Schmidt process
    ------------------------------------------------
    V - projection matrix V -> NumpyVectorArray -> V.shape = (n, r)
    W - projection matrix W -> NumpyVectorArray -> W.shape = (n, r)
    '''
    
    # Solution arrays containing len(validation_set) many reduced samples
    card_mu = len(mu)
    reduced_solution_V = pod_rom_V.solution_space.empty()
    reduced_solution_W = pod_rom_W.solution_space.empty()
    for s in mu:
        reduced_solution_V.append(pod_rom_V.solve(s))
        reduced_solution_W.append(pod_rom_W.solve(s))
        
    # It would be better to get matrices where columns are the reconstructed reduced solutions as in theory we will use such matrix; however PyMor only has vstack option (appending as a row of a matrix)
    reduced_solution_reconstruct_V_T = pod_reductor_V.reconstruct(reduced_solution_V) # a matrix with rows representing the reconstructed reduced solutions for different parameter values to first parametrized coercive model (row i will give us (s_{i}I - A)^{-1}B)
    reduced_solution_reconstruct_W_T = pod_reductor_W.reconstruct(reduced_solution_W) # a matrix with rows representing the reconstructed reduced solutions for different parameter values to second parametrized coercive model (row i will give us (s_{i}I - A)^{-*}C^T)

    # To align with the theory, we take the transpose of the result. Also, note that the transpose operation does not exist in PyMor for `NumpyVectorArray`, so we first take the transpose of the NumPy array and then convert it back
    space_V_numpy = NumpyVectorSpace(card_mu)
    space_W_numpy = NumpyVectorSpace(card_mu)
    R_V = space_V_numpy.make_array(reduced_solution_reconstruct_V_T.to_numpy().T)
    R_W = space_W_numpy.make_array(reduced_solution_reconstruct_W_T.to_numpy().T)

    R_V, R_W = R_V.to_numpy(), R_W.to_numpy() # Note: One may also use R_V.impl._array to get np.ndarray type needed for computation
    D_b, D_c = np.diag(b), np.diag(c)

    V_numpy = np.matmul(R_V, D_b)
    W_numpy = np.matmul(R_W, D_c)

    if biorth is True:
        space = NumpyVectorSpace(V_numpy.shape[0])
        V = space.make_array(V_numpy.T)
        W = space.make_array(W_numpy.T)
        [V_bi, W_bi] = gram_schmidt_biorth(V, W, check_tol = 10**(-8)) # NumpyVectorArray
        if numpy is True:
            return [V_bi.to_numpy().T, W_bi.to_numpy().T]
        else:
            return [V_bi, W_bi]       
    else:
        if numpy is True:
            return [V_numpy, W_numpy]
        else:
            space = NumpyVectorSpace(V_numpy.shape[1])
            V = space.make_array(V_numpy)
            W = space.make_array(W_numpy)
            return [V, W]

In [15]:
def error(A, B, C, mu, b, c, V_pod, W_pod, norm, biorth = None):
    
    '''
    Inputs:
    ------------------------------------------------
    A - matrix -> NumPy array or NumpyMatrixOperator -> A.shape = (n, n)
    B - (column) vector -> NumPy array or NumpyMatrixOperator -> B.shape = (n, 1)
    C - (row) vector -> NumPy array or NumpyMatrixOperator -> C.shape = (1, n)
    mu - list of -mu_i values -> type(mu[i]) = pymor.parameters.base.Mu (list of Mu objects)
    b - NumPy array -> b.shape = (r,) where r = len(mu)
    c - NumPy array -> c.shape = (r,) where r = len(mu)
    V_pod - POD-projection matrix V -> NumpyVectorArray -> V.shape = (n, r)
    W_pod - POD-projection matrix W -> NumpyVectorArray -> V.shape = (n, r)
    norm - see 'https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html' for eight different matrix norms
    ------------------------------------------------
    Output:
    ------------------------------------------------
    error - [||V_exact - V_POD||_{norm}, ||W_exact - W_POD||_{norm}] -> list of floats
    '''
    
    dim = A.shape[0] if isinstance(A, np.ndarray) else to_matrix(A).shape[0]
    
    # NumPy convertions
    A = A if isinstance(A, np.ndarray) else to_matrix(A).toarray()
    B = B if isinstance(B, np.ndarray) else to_matrix(B)
    C = C if isinstance(C, np.ndarray) else to_matrix(C)
    mu_values = np.array([s['s'] for s in mu]) 
    
    if isinstance(V_pod, np.ndarray) is False:
        V_pod_numpy = V_pod.to_numpy()
    else:
        V_pod_numpy = V_pod
    if isinstance(W_pod, np.ndarray) is False:
        W_pod_numpy = W_pod.to_numpy()
    else:
        W_pod_numpy = W_pod
    
    # Exact projection matrices
    identity = np.eye(dim)
    D_b, D_c = np.diag(b), np.diag(c)
    
    V_exact = np.matmul(np.hstack([np.matmul(np.linalg.inv(s*identity - A), B) for s in mu_values]), D_b)
    W_exact = np.matmul(np.hstack([(np.matmul(np.conjugate(np.linalg.inv(s*identity - A).T), C.T)) for s in mu_values]), D_c)

    if biorth is True:
        space = NumpyVectorSpace(V_exact.shape[0])
        V = space.make_array(V_exact.T)
        W = space.make_array(W_exact.T)
        [V_bi, W_bi] = gram_schmidt_biorth(V, W, check_tol = 10**(-8))
        V_exact = V_bi.to_numpy().T
        W_exact = W_bi.to_numpy().T

    # Computing error norms
    error_V = np.linalg.norm(V_exact - V_pod_numpy, norm)
    error_W = np.linalg.norm(W_exact - W_pod_numpy, norm)

    return [error_V, error_W]   

In [16]:
from pymor.models.examples import penzl_example

penzl = penzl_example()

A = penzl.A
B = penzl.B
C = penzl.C

# Create Matrix induced Stationary Models
[model_penzl_V, model_penzl_W] = MatrixModel(penzl.A, penzl.B, penzl.C)

# Define a parameter space
parameter_space = model_penzl_V.parameters.space(0.01, 10.)

# Create POD-Galerkin RB reductors
training_set = parameter_space.sample_randomly(30)
[pod_rom_V, pod_reductor_V, pod_rom_W, pod_reductor_W] = MatrixReductor(model_penzl_V, model_penzl_W, training_set, reduced_order_V = 20, reduced_order_W = 20)

# Find projection matrices V, W
b = np.random.rand(100)
c = np.random.rand(100)
mu = parameter_space.sample_randomly(100)
[V, W] = ProjectionMatrices(pod_rom_V = pod_rom_V, pod_reductor_V = pod_reductor_V, pod_rom_W = pod_rom_W, pod_reductor_W = pod_reductor_W, mu = mu, b = b, c = c, numpy = True)
[V_bi, W_bi] = ProjectionMatrices(pod_rom_V = pod_rom_V, pod_reductor_V = pod_reductor_V, pod_rom_W = pod_rom_W, pod_reductor_W = pod_reductor_W, mu = mu, b = b, c = c, numpy = True, biorth = True)

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

In [17]:
diag = np.diag(np.matmul(W.T, V))
diag_bi = np.diag(np.matmul(W_bi.T, V_bi))
print(f'The diagonal entries of W^TV is \n {diag} \n and diagonal entries of biorthonormalized W^TV is \n {diag_bi}.')

The diagonal entries of W^TV is 
 [2.87937836e-01 1.07297839e-02 1.79050308e-02 1.26912469e-01
 3.64502039e-02 2.55741596e-02 1.96167575e-03 3.15698704e-02
 1.73619387e-02 4.74775877e-02 6.29967180e-02 1.07675377e-02
 1.37631855e-01 4.32501757e-02 1.16889393e-01 6.39801743e-01
 4.89819415e-03 3.60855284e-02 1.98453227e-03 2.79937202e-02
 3.72932894e-04 7.99645421e-03 9.69102990e-03 1.09136495e-02
 7.97025203e-02 3.88520740e-03 2.00587138e-02 3.29471322e-02
 3.87943072e-03 6.83594004e-02 5.73901807e-02 1.26258147e-01
 3.95192259e-01 6.55322482e-02 3.00426640e-01 1.54664965e-01
 5.39945469e-03 2.91072390e-02 3.14967313e-01 1.54715215e-01
 5.43154576e-03 2.09740147e-02 1.50584569e-02 6.93064007e-02
 2.00305811e-02 6.74713710e-02 3.23574789e-01 8.01598813e-02
 3.09694926e-01 1.00724751e-02 3.48597223e-02 6.79542386e-03
 3.44705029e-01 5.50141064e-02 9.88422394e-03 2.48188702e-02
 4.21690009e-03 9.80242260e-02 8.10429047e-03 2.79840902e-02
 1.81232154e-01 1.06075879e-02 8.39731194e-03 3.919

In [18]:
# Use error function defined above
linf = error(A = A, B = B, C = C, mu = mu, b = b, c = c, V_pod = V, W_pod = W, norm = np.inf)
l2 = error(A = A, B = B, C = C, mu = mu, b = b, c = c, V_pod = V, W_pod = W, norm = 2)
linf_bi = error(A = A, B = B, C = C, mu = mu, b = b, c = c, V_pod = V_bi, W_pod = W_bi, norm = np.inf, biorth = True)
l2_bi = error(A = A, B = B, C = C, mu = mu, b = b, c = c, V_pod = V_bi, W_pod = W_bi, norm = 2, biorth = True)
print(f'The Linf norm of V - V_POD is {linf[0]} and W - W_POD is {linf[1]}.')
print(f'The L2 norm of V - V_POD is {l2[0]} and W - W_POD is {l2[1]}.')
print(f'The Linf norm of V^bi - V^bi_POD is {linf_bi[0]} and W^bi - W^bi_POD is {linf_bi[1]}.')
print(f'The L2 norm of V^bi - V^bi_POD is {l2_bi[0]} and W^bi - W^bi_POD is {l2_bi[1]}.')

Accordion(children=(HTML(value='', layout=Layout(height='16em', width='100%')),), titles=('Log Output',))

The Linf norm of V - V_POD is 1.8690843144042457e-13 and W - W_POD is 2.1727758481304704e-13.
The L2 norm of V - V_POD is 3.015086157364575e-14 and W - W_POD is 3.319540434996445e-14.
The Linf norm of V^bi - V^bi_POD is 7404.780079656258 and W^bi - W^bi_POD is 11.746076891631116.
The L2 norm of V^bi - V^bi_POD is 16988.256279859612 and W^bi - W^bi_POD is 4.528500696152019.
